In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd

data = pd.read_csv("data.csv",delimiter="\t")

In [4]:
data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P7,P8,P9,P10,age,gender,accuracy,country,source,elapsed
0,1,4,2,3,3,2,3,4,4,3,...,1,2,5,2,17,1,92,US,6,914
1,4,3,4,3,4,4,4,4,2,2,...,3,2,2,2,37,1,100,US,1,891
2,3,4,4,4,4,4,4,3,2,2,...,2,3,3,3,31,1,80,US,6,903
3,4,5,4,4,4,3,3,2,2,2,...,2,3,4,4,32,1,93,US,1,806
4,4,0,4,4,4,3,5,1,2,4,...,4,2,3,2,46,2,87,NZ,1,1826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,24,1,100,IN,1,274
49155,4,5,4,4,4,3,5,3,2,3,...,3,4,4,4,22,1,50,ZA,1,584
49156,2,4,2,3,3,3,4,4,2,4,...,3,4,5,4,29,2,85,BE,6,932
49157,4,4,4,4,4,4,4,4,1,3,...,2,4,5,4,48,1,95,CZ,2,1212


In [7]:
import pandas as pd

# Get the "country" column
countries = data['country']

# Get unique countries and assign unique numbers
unique_countries = countries.unique()
country_dict = {country: i for i, country in enumerate(unique_countries)}

# Replace countries with unique numbers
data['country'] = countries.replace(country_dict)


In [8]:
data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,P7,P8,P9,P10,age,gender,accuracy,country,source,elapsed
0,1,4,2,3,3,2,3,4,4,3,...,1,2,5,2,17,1,92,0,6,914
1,4,3,4,3,4,4,4,4,2,2,...,3,2,2,2,37,1,100,0,1,891
2,3,4,4,4,4,4,4,3,2,2,...,2,3,3,3,31,1,80,0,6,903
3,4,5,4,4,4,3,3,2,2,2,...,2,3,4,4,32,1,93,0,1,806
4,4,0,4,4,4,3,5,1,2,4,...,4,2,3,2,46,2,87,1,1,1826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,24,1,100,6,1,274
49155,4,5,4,4,4,3,5,3,2,3,...,3,4,4,4,22,1,50,11,1,584
49156,2,4,2,3,3,3,4,4,2,4,...,3,4,5,4,29,2,85,40,6,932
49157,4,4,4,4,4,4,4,4,1,3,...,2,4,5,4,48,1,95,39,2,1212
